In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import netCDF4 as nc
import xarray as xr
import scipy.ndimage
import scipy as sp
import alphashape
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')


In [15]:
# Read in data

data = pd.read_csv('../Data/DataFromInka/Princess_Ragnild_Coast_airborne_IRHs_20190107_02_depth20220801_working_copy.txt', sep=" ")

data

,year_acq,month_acq,day_acq,surveyID,profileID,lon,lat,psX,psY,trace,...,IRH3,IRH4,IRH5,IRH6,IRH7,IRH8,IRH9,IRH10,IRH11,IRH12
0,2019,1,7,1,2,26.160190,-70.128927,961154.51,1956754.7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,1,7,1,2,26.160348,-70.128932,961159.69,1956751.7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,1,7,1,2,26.160506,-70.128936,961164.87,1956748.6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,1,7,1,2,26.160663,-70.128940,961170.05,1956745.6,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,1,7,1,2,26.160821,-70.128944,961175.23,1956742.5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51197,2019,1,7,1,10,25.605531,-70.279899,934870.77,1950742.5,9644,...,13.5,19.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51198,2019,1,7,1,10,25.605373,-70.279891,934865.73,1950745.9,9645,...,13.2,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51199,2019,1,7,1,10,25.605214,-70.279883,934860.69,1950749.2,9646,...,13.2,18.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51200,2019,1,7,1,10,25.605055,-70.279876,934855.64,1950752.6,9647,...,13.2,18.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Clean up dataset: remove data not needed

data.drop(['year_acq', 'month_acq', 'day_acq', 'surveyID', 'profileID', 'lon', 'lat', 'trace', 'year_pick', 'month_pick', 'day_pick' ],axis=1, inplace=True)


In [17]:
# Add columns for the corresponding density due to the density profile

data['IRH0'] = [0] * len(data)
data['AdjDepthIRH0'] = [0] * len(data)

for i in range(1, 13):
    data['DensIRH' + str(i)] = [0] * len(data)
    
# Add columns for adjusted depth

for i in range(1, 13):
    data['AdjDepthIRH' + str(i)] = data['IRH' + str(i)]
    

In [18]:
data

,psX,psY,surface,base,IRH1,IRH2,IRH3,IRH4,IRH5,IRH6,...,AdjDepthIRH3,AdjDepthIRH4,AdjDepthIRH5,AdjDepthIRH6,AdjDepthIRH7,AdjDepthIRH8,AdjDepthIRH9,AdjDepthIRH10,AdjDepthIRH11,AdjDepthIRH12
0,961154.51,1956754.7,0,178.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,961159.69,1956751.7,0,178.4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,961164.87,1956748.6,0,178.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,961170.05,1956745.6,0,178.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,961175.23,1956742.5,0,179.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51197,934870.77,1950742.5,0,327.3,NaN,8.6,13.5,19.3,NaN,NaN,...,13.5,19.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51198,934865.73,1950745.9,0,328.0,NaN,8.9,13.2,19.0,NaN,NaN,...,13.2,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51199,934860.69,1950749.2,0,328.0,NaN,8.9,13.2,18.7,NaN,NaN,...,13.2,18.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51200,934855.64,1950752.6,0,329.8,NaN,8.9,13.2,18.7,NaN,NaN,...,13.2,18.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# This code calculates the density adjustment. You can choose the target density and the 
# observed density profile

# The function takes as input a depth and returns the density at that depth. The current 
# function describes the density profile of an ice core from Derwael Ice Rise. See
# doi:10.1002/2013GL058023 .

def DensityProfile(d):
    return 910.0 - 460.0 * np.exp(-0.025*d)

# Calculate the density for each point:

for i in range(1,13):
    for j in range(2000):
        if not np.isnan(data.loc[j,'IRH' + str(i)]):
            data.loc[j,'DensIRH' + str(i)] = DensityProfile(data.loc[j,'IRH' + str(i)])

# This code calculates the equivalent depth after a density adjustment assuming a uniform
# density of 900 kg m^{-3}. These calculations allow for an adjustment where NaN values appear.
            
for i in range(1,13):
    for j in range(2000):
        if not np.isnan(data.loc[j,'AdjDepthIRH' + str(i)]):
            k = i
            while np.isnan(data.loc[j,'AdjDepthIRH' + str(k-1)]) and k>=0:
                k = k - 1
            a0 = data.loc[j,'AdjDepthIRH' + str(i)]
            data.loc[j,'AdjDepthIRH' + str(i)] = data.loc[j,'AdjDepthIRH' + str(k-1)] + ((data.loc[j,'AdjDepthIRH' + str(i)] - data.loc[j,'AdjDepthIRH' + str(k-1)])* data.loc[j,'DensIRH' + str(i)]) / 900.0
            a1 = data.loc[j,'AdjDepthIRH' + str(i)]
            for l in range(i+1, 13):
                data.loc[j,'AdjDepthIRH' + str(l)] = data.loc[j,'AdjDepthIRH' + str(l)] - (a0 - a1)



In [21]:
#Test against paper calculations

for i in range(1,13):
    print(data['IRH' + str(i)][1005], data['AdjDepthIRH' + str(i)][1005], data['DensIRH' + str(i)][1005])


8.3 4.944936269900348 536.197908784375
12.4 7.553512981634782 572.6144001368269
18.5 11.758002997770806 620.3345925446594
26.0 17.340160853520246 669.8589426899326
33.7 23.430937907056805 711.9090062575195
42.6 30.861698087704585 751.4251868070787
54.2 41.06124612859909 791.3442445521603
62.7 48.74958740765991 814.0596648417338
75.2 60.413595065324245 839.8085513518334
89.1 73.70219753800423 860.4131097418692
97.4 81.7228106510826 869.7050363578935
108.7 92.76695055454088 879.621762222343


In [22]:
data

,psX,psY,surface,base,IRH1,IRH2,IRH3,IRH4,IRH5,IRH6,...,AdjDepthIRH3,AdjDepthIRH4,AdjDepthIRH5,AdjDepthIRH6,AdjDepthIRH7,AdjDepthIRH8,AdjDepthIRH9,AdjDepthIRH10,AdjDepthIRH11,AdjDepthIRH12
0,961154.51,1956754.7,0,178.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,961159.69,1956751.7,0,178.4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,961164.87,1956748.6,0,178.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,961170.05,1956745.6,0,178.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,961175.23,1956742.5,0,179.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51197,934870.77,1950742.5,0,327.3,NaN,8.6,13.5,19.3,NaN,NaN,...,13.5,19.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51198,934865.73,1950745.9,0,328.0,NaN,8.9,13.2,19.0,NaN,NaN,...,13.2,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51199,934860.69,1950749.2,0,328.0,NaN,8.9,13.2,18.7,NaN,NaN,...,13.2,18.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51200,934855.64,1950752.6,0,329.8,NaN,8.9,13.2,18.7,NaN,NaN,...,13.2,18.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
